# BFCL: Evaluate Generated Plans

In [ ]:
import osimport jsonimport requestsimport loggingimport pandas as pdfrom dotenv import load_dotenv

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')load_dotenv()OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')if not OPENAI_API_KEY: raise EnvironmentError('OPENAI_API_KEY not set')

In [ ]:
# Load generated plansif not os.path.exists('bfcl_eval_results.parquet'):    raise FileNotFoundError('bfcl_eval_results.parquet not found; run gen notebook first')bfcl_df = pd.read_parquet('bfcl_eval_results.parquet', engine='fastparquet')

In [ ]:
# Select evaluation engine: 'manual' or 'openai'EVAL_ENGINE = 'openai'if EVAL_ENGINE not in ['manual','openai']:    raise ValueError("EVAL_ENGINE must be 'manual' or 'openai'")

In [ ]:
# OpenAI Eval API wrappersdef create_eval():    url = 'https://api.openai.com/v1/evals'    headers = {        'Authorization': f'Bearer {OPENAI_API_KEY}',        'Content-Type': 'application/json'    }    data = {        'name': 'BFCL Planned Trajectory',        'data_source_config': {            'type': 'custom',            'item_schema': {                'type': 'object',                'properties': {                    'instructions': {'type': 'string'},                    'planned_trajectory': {                        'type': 'array',                        'items': {                            'type': 'object',                            'properties': {                                'name': {'type': 'string'},                                'arguments': {'type': 'object'}                            },                            'required': ['name','arguments']                        }                    },                    'output_planned_trajectory': {                        'type': 'array',                        'items': {                            'type': 'object',                            'properties': {                                'name': {'type': 'string'},                                'arguments': {'type': 'object'}                            },                            'required': ['name','arguments']                        }                    }                },                'required': ['instructions','planned_trajectory','output_planned_trajectory']            }        },        'testing_criteria': [            {                'type': 'text_similarity',                'name': 'Match output to human label',                'evaluation_metric': 'bleu',                'input': '{{ item.planned_trajectory }}',                'pass_threshold': 0.5,                'reference': '{{ item.output_planned_trajectory }}'            }        ]    }    response = requests.post(url, headers=headers, json=data)    return response.json()

In [ ]:
def upload_eval_file(file_path):    url = 'https://api.openai.com/v1/files'    headers = {'Authorization': f'Bearer {OPENAI_API_KEY}'}    files = { 'file': open(file_path,'rb'), 'purpose': (None,'evals') }    response = requests.post(url, headers=headers, files=files)    return response.json()

In [ ]:
def create_eval_run(eval_id, file_id):    url = f'https://api.openai.com/v1/evals/{eval_id}/runs'    headers = {'Authorization': f'Bearer {OPENAI_API_KEY}', 'Content-Type': 'application/json'}    data = {'data_source': {'type':'jsonl','source':{'type':'file_id','id':file_id}}}    response = requests.post(url, headers=headers, json=data)    return response.json()

In [ ]:
def get_eval_run(eval_id, run_id):    url = f'https://api.openai.com/v1/evals/{eval_id}/runs/{run_id}'    headers = {'Authorization': f'Bearer {OPENAI_API_KEY}', 'Content-Type':'application/json'}    response = requests.get(url, headers=headers)    return response.json()

In [ ]:
# Format eval data to JSONLdef format_eval_data(df, output_file):    with open(output_file,'w') as f:        for _, row in df.iterrows():            obj = { 'item': { 'instructions': row['instruction'], 'planned_trajectory': row['ground_truth_calls'], 'output_planned_trajectory': row['bfcl_planned_calls'] } }            f.write(json.dumps(obj) + '
')format_eval_data(bfcl_df, 'bfcl_eval_data.jsonl')

# To run OpenAI eval:
# eval_meta = create_eval()
# file_resp = upload_eval_file('bfcl_eval_data.jsonl')
# eval_run = create_eval_run(eval_meta['id'], file_resp['id'])
# get_eval_run(eval_meta['id'], eval_run['id'])

# Quick manual inspection

In [ ]:
bfcl_df[['instruction','ground_truth_calls','bfcl_planned_calls']].head(10)